# Spatial transcriptomics analysis

- Reference: https://www.sc-best-practices.org/cellular_structure/annotation.html

In [21]:
import scanpy as sc
import squidpy as sq
import SpatialDE
import NaiveDE
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import math
import seaborn as sns
from scipy.stats import median_abs_deviation
import scipy

In [ ]:
import anndata2ri
import logging
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

from rpy2.robjects.conversion import localconverter

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()

%load_ext rpy2.ipython

In [23]:
#sc.logging.print_versions()
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 3
out_folder = "results"

In [24]:
# parameters
sample_ids = ["mesom03", "mesom26"]

In [ ]:
adata03 = sc.read_visium('visium/mesom03/outs')

In [ ]:
adata26 = sc.read_visium('visium/mesom26/outs')

In [ ]:
"CHST4" in adata26.var_names

In [ ]:
sum(adata03.X.data)

some hub genes from Nicolas's paper:
test_genes = {
  "Carcinoid_A1": ["DLL3", "ASCL1"],
  "Carcinoid_A2": ["ROBO1", "SLIT1"],
  "Supraca": ["PD-L1"],
  "LCNEC": ["ANGPTL3", "OTP", "NKX2-1", "ERBB4"]
}

in our case : genes for each of the 3 archetypes

## Import data & QC

In [29]:
def find_outliers(dat):
    q3, q1 = np.percentile(dat, [75 ,25])
    IQR = q3-q1
    lower_bound, upper_bound = np.percentile(dat, [1 ,99])
    return q3,q1,lower_bound,upper_bound

In [30]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [ ]:
#ddir = os.path.join("data", "nicolas_spatial_data")
ddir = os.path.join("/data/mesomics/work/mesomics2/mangel", "visium")
adatas = []
adatas_copy = []

fig, axes = plt.subplots(2, 6, figsize=(24, 8))
axes = axes.flatten()

fig2, axes2 = plt.subplots(2, 4, figsize=(24, 8))
axes2 = axes2.flatten()

"""
fig3, axes3 = plt.subplots(2, 2, figsize=(12, 8))
axes3 = axes3.flatten()
"""

for i in range(len(sample_ids)):
    sample_id = sample_ids[i]
    sample_ddir = os.path.join(ddir, sample_id, "outs")
    adata = sc.read_visium(sample_ddir)
    adata.var_names_make_unique()
    adata.obs['sample'] = sample_id
    clusters_path = os.path.join(sample_ddir, "analysis/clustering/gene_expression_graphclust","clusters.csv")
    clusters_df = pd.read_csv(clusters_path)
    # Now assign the values to adata.obs['cluster']
    adata.obs['cluster'] = clusters_df.set_index('Barcode').loc[adata.obs.index, clusters_df.columns[1]].values
    print(adata)
    
    # mitochondrial genes
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    # ribosomal genes
    adata.var["ribo"] = adata.var_names.str.startswith("RP[SL]")
    # calculate QC metrics
    sc.pp.calculate_qc_metrics(
        adata, 
        qc_vars=["mt", "ribo"], 
        percent_top = (20, 50, 100, 200, 500),
        inplace=True
    )
    
    # identify outlier via MAD
    adata.obs["outlier"] = (
        is_outlier(adata, "log1p_total_counts", 5)
        | is_outlier(adata, "log1p_n_genes_by_counts", 5)
        | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
    )
    adata.obs.outlier.value_counts()
    
    # identify outlier via MT gene pct
    # !!! percentage of MT gene is all 0
    adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 5) | (
        adata.obs["pct_counts_mt"] > 20
    )
    adata.obs.mt_outlier.value_counts()
    
    # visualize
    q3,q1,tc_lower_bound,tc_upper_bound = find_outliers(adata.obs["total_counts"])
    q3_,q1_,tc_lower_bound_,tc_upper_bound_ = find_outliers(adata.obs["log1p_total_counts"])
    
    
    sns.distplot(adata.obs["total_counts"], kde=False, ax=axes[6*i])
    axes[6*i].axvline(tc_lower_bound, color='red')
    axes[6*i].axvline(tc_upper_bound, color='red')
    axes[6*i].set_title(f'before filtering', fontsize=10)
    axes[6*i].set_ylabel(sample_id)
    
    """
    sns.distplot(adata.obs["log1p_total_counts"], kde=False, ax=axes[6*i])
    axes[6*i].axvline(tc_lower_bound_, color='red')
    axes[6*i].axvline(tc_upper_bound_, color='red')
    axes[6*i].set_title(f'before filtering', fontsize=10)
    axes[6*i].set_ylabel(sample_id)
    """
    sc.pl.scatter(
        adata, 
        "n_genes_by_counts", 
        "pct_counts_in_top_20_genes", 
        color="pct_counts_mt",
        ax=axes[6*i+2],
        #colorbar=False,
        #colorbar_ax=axes[5*(i+1)],
        #colorbar_ax=axes[6*i+3], 
        show = False,
        legend_loc='none'
    )
    
    # Manual colorbar creation
    # Get the scatter plot's norm and cmap
    scatter = axes[6*i+2].collections[0]  # Get the scatter plot collection
    norm = scatter.norm
    cmap = scatter.get_array()  # Get the colormap

    # Create a colorbar on the last column axis
    cbar = fig.colorbar(scatter, ax=axes[6*i+2], orientation='vertical')
    cbar.set_label("Percentage Counts MT")
    
    sc.pl.scatter(
        adata, 
        "n_genes_by_counts", 
        "pct_counts_in_top_20_genes", 
        color="outlier",
        ax=axes[6*i+3], 
        show = False,
        legend_loc='none'
    )
    df = adata.obs[['sample', 'total_counts', 'outlier']]
    sns.violinplot(
        data=df,
        x='sample',
        y='total_counts',
        hue='outlier',
        split=True,
        scale='count',
        inner='quartile',
        ax=axes[6*i+4]
    )
    
    # for spatial visualization of outlier spots
    adatas_copy.append(adata.copy())
    """ TO BE USED WHEN COLORBAR LOCATION PB IS FIXED (for now appears on wrong ax)
    adatas_copy[i].obsm['spatial'] = adatas_copy[i].obsm['spatial'].astype('float')
    adatas_copy[i].obs['outlier'] = adatas_copy[i].obs['outlier'].astype('category')
    adatas_copy[i].obs['mt_outlier'] = adatas_copy[i].obs['mt_outlier'].astype('category')
    sc.pl.spatial(adatas_copy[i], color='outlier', scale_factor=1.5, ax=axes3[2*i], show=False)
    sc.pl.spatial(adatas_copy[i], color='mt_outlier', scale_factor=1.5, ax=axes3[2*i+1], show=False)
    axes3[2*i].set_title("Outliers")
    axes3[2*i+1].set_title("MT Outliers")
    """
    
    # filtering
    print(f"Total number of spots: {adata.n_obs}")
    print(f"Total number of reads: {sum(adata.X.data)}")
    adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()
    print(f"Number of spots after filtering of low quality spots: {adata.n_obs}")
    print(f"Number of reads after filtering of low quality spots: {sum(adata.X.data)}")

    adata.obs_names.to_series().to_csv("scanpy_selected_cells_{}_06052024.txt".format(sample_id))
    
    sns.distplot(adata.obs["total_counts"], kde=False, ax=axes[6*i+1])
    axes[6*i+1].axvline(tc_lower_bound_, color='red')
    axes[6*i+1].axvline(tc_upper_bound_, color='red')
    axes[6*i+1].set_title(f'after filtering', fontsize=10)

    adata.layers["counts"] = adata.X.copy()
    
    # normalization
    scales_counts = sc.pp.normalize_total(adata, target_sum=None, inplace=False)
    # log1p transform
    adata.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)
    sns.histplot(
        adata.obs["total_counts"], 
        bins=100, 
        kde=False, 
        ax=axes2[4*i]
    )
    axes2[4*i].set_ylabel(sample_id)
    axes2[4*i].set_title("Total counts")
    
    scales_counts = sc.pp.normalize_total(adata, target_sum=None, inplace=False)
    # log1p transform
    adata.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)
    sns.histplot(
        adata.layers["log1p_norm"].sum(1), 
        bins=100, 
        kde=False, 
        ax=axes2[4*i+1]
    )
    axes2[4*i+1].set_title("Shifted logarithm")
    
    adatas.append(adata)
    

fig.tight_layout()

fig.savefig(os.path.join('ST_QC.png'))
fig2.savefig(os.path.join('ST_norm.png'))
#fig3.savefig(os.path.join('ST_outliers.png'))

fig.show()

In [ ]:
adatas[1].obsm["spatial"]

In [ ]:
set(adatas[0].obs["outlier"])

In [15]:
adatas[0].write("sample_3B/sample3B_visium_QC_norm.h5ad")
adatas[1].write("sample_26/sample26_visium_QC_norm.h5ad")

In [ ]:
fig3, axes3 = plt.subplots(2, 2, figsize=(12, 8))
axes3 = axes3.flatten()

for i in range(len(adatas_copy)):

    adatas_copy[i].obsm['spatial'] = adatas_copy[i].obsm['spatial'].astype('float')
    adatas_copy[i].obs['outlier'] = adatas_copy[i].obs['outlier'].astype('category')
    adatas_copy[i].obs['mt_outlier'] = adatas_copy[i].obs['mt_outlier'].astype('category')
    adatas_copy[i].obs['cluster'] = adatas_copy[i].obs['cluster'].astype('category')
    sc.pl.spatial(adatas_copy[i], color='outlier', scale_factor=1.5, ax=axes3[2*i], show=False)
    sc.pl.spatial(adatas_copy[i], color='mt_outlier', scale_factor=1.5, ax=axes3[2*i+1], show=False)
    axes3[2*i].set_title("Outliers")
    axes3[2*i+1].set_title("MT Outliers")

fig3.tight_layout()
fig3.savefig(os.path.join('ST_outliers.png'))
fig3.show()

In [ ]:
sq.pl.spatial_scatter(adatas_copy[0], color="cluster", figsize=(6, 6))
sq.pl.spatial_scatter(adatas_copy[1], color="cluster", figsize=(6, 6))

Do same modifications to actual adata objects:

In [18]:
for i in range(len(adatas)):

    adatas[i].obsm['spatial'] = adatas[i].obsm['spatial'].astype('float')
    adatas[i].obs['outlier'] = adatas[i].obs['outlier'].astype('category')
    adatas[i].obs['mt_outlier'] = adatas[i].obs['mt_outlier'].astype('category')
    adatas[i].obs['cluster'] = adatas[i].obs['cluster'].astype('category')

In [ ]:
sq.pl.spatial_scatter(adatas[0], color="cluster", figsize=(6, 6))
sq.pl.spatial_scatter(adatas[1], color="cluster", figsize=(6, 6))

# Mapping & deconvolution : Cell2location

In [34]:
import cell2location as c2l
import matplotlib

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor="white")

In [4]:
adata_sc_3B = sc.read("sample_3B/sample3B_annotation.h5ad")
adata_sc_26 = sc.read("sample_26/sample26_annotation.h5ad")

In [ ]:
list(cell_type for cell_type in set(adata_sc_3B.obs["celltypist_cell_label_coarse"]) if cell_type not in ["ILC","pDC","Monocytes","Epithelial cells","DC"])

In [13]:
adatas3B = sc.read("sample_3B/sample3B_visium_QC_norm.h5ad")
adatas26 = sc.read("sample_26/sample26_visium_QC_norm.h5ad")

In [16]:
adatas = [adatas3B, adatas26]

In [ ]:
adatas_sc = [adata_sc_3B, adata_sc_26]

In [10]:
def deconvol(adata_sc_ref,adata_st,list_celltypes):
    
    adata_sc = adata_sc_ref.copy()
    adata = adata_st.copy()
    
    sq.pl.spatial_scatter(adata)
    
    adata_sc.X = adata_sc.layers["counts"]
    sc.pl.umap(adata_sc, color="celltypist_cell_label_coarse")
    
    adata.var["feature_name"] = adata.var_names
    adata.var.set_index("gene_ids", drop=True, inplace=True)
    adata_sc.var["feature_name"] = adata_sc.var_names
    adata_sc.var.set_index("gene_ids", drop=True, inplace=True)
    
    # find mitochondrial (MT) genes
    adata.var["MT_gene"] = [
        gene.startswith("MT-") for gene in adata.var["feature_name"]
    ]
    # remove MT genes for spatial mapping (keeping their counts in the object)
    adata.obsm["MT"] = adata[:, adata.var["MT_gene"].values].X.toarray()
    adata = adata[:, ~adata.var["MT_gene"].values]
    
    shared_features = [
        feature for feature in adata.var_names if feature in adata_sc.var_names
    ]
    adata_sc = adata_sc[:, shared_features].copy()
    adata = adata[:, shared_features].copy()
    
    ### Fitting reference model
    # select common genes
    selected = c2l.utils.filtering.filter_genes(
        adata_sc, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12
    )
    adata_sc = adata_sc[:, selected].copy()
    adata = adata[:, selected].copy()
    
    # create and train ref model
    c2l.models.RegressionModel.setup_anndata(
        adata=adata_sc,
        labels_key="celltypist_cell_label_coarse",
        layer="counts",
    )
    model = c2l.models.RegressionModel(adata_sc)
    model.train(max_epochs=400, batch_size=2500, train_size=1, lr=0.002) #use_gpu=True
    print(model.plot_history(20))
    
    model.export_posterior(
        adata_sc,
        sample_kwargs={"num_samples": 1000, "batch_size": 2500},
    )
    
    model.plot_QC()
    
    # export estimated expression in each cluster
    if "means_per_cluster_mu_fg" in adata_sc.varm.keys():
        inf_aver = adata_sc.varm["means_per_cluster_mu_fg"][
            [f"means_per_cluster_mu_fg_{i}" for i in adata_sc.uns["mod"]["factor_names"]]
        ].copy()
    else:
        inf_aver = adata_sc.var[
            [f"means_per_cluster_mu_fg_{i}" for i in adata_sc.uns["mod"]["factor_names"]]
        ].copy()

    inf_aver.columns = adata_sc.uns["mod"]["factor_names"]
    inf_aver.head()
    
    ### Cell type mapping
    # prepare data
    c2l.models.Cell2location.setup_anndata(
    adata=adata
    )
    
    model = c2l.models.Cell2location(
    adata,
    cell_state_df=inf_aver,
    N_cells_per_location=8,
    )
    model.view_anndata_setup()
    
    # train mapping model
    
    model.train(max_epochs=30000, batch_size=None, train_size=1) #use_gpu=True
    # plot training history
    print(model.plot_history())
    
    adata = model.export_posterior(
    adata,
    sample_kwargs={
        "num_samples": 1000,
        "batch_size": model.adata.n_obs,
        },
    )
    
    model.plot_QC()
    
    # visualize
    adata.obs[adata.uns["mod"]["factor_names"]] = adata.obsm[
    "q05_cell_abundance_w_sf"
    ]
    # select one slide for visualization

    with matplotlib.rc_context({"figure.figsize": [4.5, 5]}):
        sc.pl.spatial(
            adata,
            cmap="magma",
            color=adata.uns["mod"]["factor_names"],
            ncols=4,
            size=1.3,
            img_key="hires",
            # limit color scale at 99.2% quantile of cell abundance
            vmin=0,
            vmax="p99.2",
        )
    
    clust_col = list_celltypes
    clust_labels = clust_col

    with matplotlib.rc_context({"figure.figsize": (15, 15)}):
        fig = c2l.plt.plot_spatial(
            adata=adata,
            color=clust_col,
            labels=clust_labels,
            max_color_quantile=0.992,
            circle_diameter=6,
            show_img=True,
            colorbar_position="right",
            colorbar_shape={"horizontal_gaps": 0.2},
        )
    
    return adata, adata_sc

In [ ]:
adata3B_deconvol, adata_sc_3B_deconvol = deconvol(adata_sc_3B,adatas3B,list(cell_type for cell_type in set(adata_sc_3B.obs["celltypist_cell_label_coarse"]) if cell_type not in ["ILC","pDC","Monocytes","Epithelial cells","DC"]))

import matplotlib.pyplot as plt
%matplotlib inline

adata_sc = sc.read("sample_3B/sample3B_sc_ref_deconvol.h5ad")
adatas3B = sc.read("sample_3B/sample3B_visium_deconvol.h5ad")

clust_col = list(cell_type for cell_type in set(adata_sc.obs["celltypist_cell_label_coarse"]) if cell_type not in ["ILC","pDC","Monocytes","Epithelial cells","DC"])
clust_labels = clust_col

with matplotlib.rc_context({"figure.figsize": (15, 15)}):
    fig = c2l.plt.plot_spatial(
        adata=adatas3B,
        color=clust_col,
        labels=clust_labels,
        max_color_quantile=0.992,
        circle_diameter=6,
        show_img=True,
        colorbar_position="right",
        colorbar_shape={"horizontal_gaps": 0.2},
    )

plt.savefig("sample_3B/deconv_c2l_3B.svg", format="svg")
plt.show()

In [12]:
adata3B_deconvol.write("sample_3B/sample3B_visium_deconvol.h5ad")
adata_sc_3B_deconvol.write("sample_3B/sample3B_sc_ref_deconvol.h5ad")

In [ ]:
adata26_deconvol, adata_sc_26_deconvol = deconvol(adata_sc_26,adatas26,list(set(adata_sc_26.obs["celltypist_cell_label_coarse"])))

In [16]:
adata26_deconvol.write("sample_26/sample26_visium_deconvol.h5ad")
adata_sc_26_deconvol.write("sample_26/sample26_sc_ref_deconvol.h5ad")

## Spatial neighborhoods & domains

In [ ]:
for i in range(len(adatas)):
    sq.gr.spatial_neighbors(adatas[i])
    sq.gr.nhood_enrichment(adatas[i], cluster_key="cluster")
    sq.pl.nhood_enrichment(adatas[i], cluster_key="cluster", method="average", figsize=(5, 5))
    sq.gr.interaction_matrix(adatas[i], cluster_key="cluster")
    sq.pl.interaction_matrix(adatas[i], cluster_key="cluster", method="average", figsize=(5, 5))
    
    # nearest neighbor graph
    sc.pp.neighbors(adatas[i])
    nn_graph_genes = adatas[i].obsp["connectivities"]
    # spatial proximity graph
    sq.gr.spatial_neighbors(adatas[i])
    nn_graph_space = adatas[i].obsp["spatial_connectivities"]

    alpha = 0.2
    joint_graph = (1 - alpha) * nn_graph_genes + alpha * nn_graph_space
    sc.tl.leiden(adatas[i], adjacency=joint_graph, key_added="squidpy_domains")
    
    sq.pl.spatial_scatter(adatas[i], color=["cluster", "squidpy_domains"], wspace=0.9)

In [ ]:
"KDR" in adatas[0].var_names

# Gene variation (HVG) in space : Squidpy

In [ ]:
adatas_norm_copy = adatas.copy()

for i in range(len(adatas)):

    adatas_norm_copy[i].X = adatas_norm_copy[i].layers["log1p_norm"]
    
    sq.gr.spatial_neighbors(adatas_norm_copy[i])
    sq.gr.spatial_autocorr(adatas_norm_copy[i], mode="moran", genes=adatas_norm_copy[i].var_names)
    
    adatas_norm_copy[i].uns["moranI"]
   
    sq.pl.spatial_scatter(adatas_norm_copy[i], color=["CTLA4","CD274","BAP1","CHST4"]) #list(top10["I"][:1])+["CTLA4","CD274","BAP1"]

In [ ]:
adatas_norm_copy = adatas.copy()

for i in range(len(adatas)):

    adatas_norm_copy[i].X = adatas_norm_copy[i].layers["log1p_norm"]
    
    sq.gr.spatial_neighbors(adatas_norm_copy[i])
    sq.gr.spatial_autocorr(adatas_norm_copy[i], mode="moran", genes=adatas_norm_copy[i].var_names)
    
    adatas_norm_copy[i].uns["moranI"]
   
    sq.pl.spatial_scatter(adatas_norm_copy[i], color=["FLT1","FLT4","KDR"]) #list(top10["I"][:1])+["CTLA4","CD274","BAP1"]

# Gene variation (HVG) in space : SpatialDE

In [ ]:
for i in range(len(adatas)):
    
    adatas[i].var_names_make_unique()
    
    counts = sc.get.obs_df(adatas[i], keys=list(adatas[i].var_names), use_raw=False)
    
    total_counts = sc.get.obs_df(adatas[i], keys=["total_counts"])
    
    norm_expr = NaiveDE.stabilize(counts.T).T
    
    resid_expr = NaiveDE.regress_out(total_counts, norm_expr.T, "np.log(total_counts)").T
    
    results = SpatialDE.run(adatas[i].obsm["spatial"], resid_expr)
    
    top10 = results.sort_values("qval").head(10)[["g", "l", "qval"]]
    top10
    
    sq.pl.spatial_scatter(adatas[i], color=list(top10["g"][:3]) + ["cluster"])
    sq.pl.spatial_scatter(adatas[i], color=["CTLA4", "CD274","BAP1"] + ["cluster"])
    

In [30]:
adatas[0].var_names_make_unique()

In [ ]:
adatas[0]

In [32]:
counts = sc.get.obs_df(adatas[0], keys=list(adatas[0].var_names), use_raw=False)

In [33]:
total_counts = sc.get.obs_df(adatas[0], keys=["total_counts"])

In [ ]:
norm_expr = NaiveDE.stabilize(counts.T).T

In [35]:
resid_expr = NaiveDE.regress_out(total_counts, norm_expr.T, "np.log(total_counts)").T

In [ ]:
results = SpatialDE.run(adatas[0].obsm["spatial"], resid_expr)

In [ ]:
results.head()

In [ ]:
top10 = results.sort_values("qval").head(10)[["g", "l", "qval"]]
top10

In [ ]:
sq.pl.spatial_scatter(adatas[0], color=list(top10["g"][:3]) + ["cluster"])

In [ ]:
sq.pl.spatial_scatter(adatas[0], color=["CTLA4", "CD274","BAP1"] + ["cluster"])

Gene variation using normal scRNAseq data (for QC):

In [ ]:
%%R
library(scry)

In [9]:
def hvg(binomial_deviance, axes2, i, adata):
    idx = binomial_deviance.argsort()[-4000:]
    mask = np.zeros(adata.var_names.shape, dtype=bool)
    mask[idx] = True

    adata.var["highly_deviant"] = mask
    adata.var["binomial_deviance"] = binomial_deviance
    
    sc.pp.highly_variable_genes(adata, layer="log1p_norm")
    
    axes2[2*i+2] = sns.scatterplot(
        data=adata.var, x="means", y="dispersions", hue="highly_deviant", s=5
    )
    return adata, axes2

### Sample mesom03

In [ ]:
anndata2ri.activate()
%reload_ext rpy2.ipython

In [ ]:
i = 0
sample_id = sample_ids[i]
adata = adatas[i]
print("processing sample: {}".format(sample_id))
ro.globalenv["adata"] = adata

In [13]:
%%R
sce = devianceFeatureSelection(adata, assay="X")

In [ ]:
binomial_deviance = ro.r("rowData(sce)$binomial_deviance").T
adatas[i], axes2 = hvg(binomial_deviance, axes2, i, adata)

### Sample LNEN084-IARC-B

In [ ]:
i = 1
sample_id = sample_ids[i]
adata = adatas[i]
print("processing sample: {}".format(sample_id))
ro.globalenv["adata"] = adata

In [16]:
%%R
sce = devianceFeatureSelection(adata, assay="X")

In [ ]:
binomial_deviance = ro.r("rowData(sce)$binomial_deviance").T
adatas[i], axes2 = hvg(binomial_deviance, axes2, i, adata)

### LNEN107-IARC-C

In [ ]:
i = 2
sample_id = sample_ids[i]
adata = adatas[i]
print("processing sample: {}".format(sample_id))
ro.globalenv["adata"] = adata

In [19]:
%%R
sce = devianceFeatureSelection(adata, assay="X")

In [ ]:
binomial_deviance = ro.r("rowData(sce)$binomial_deviance").T
adatas[i], axes2 = hvg(binomial_deviance, axes2, i, adata)

### LNEN206-IARC-D

In [ ]:
i = 3
sample_id = sample_ids[i]
adata = adatas[i]
print("processing sample: {}".format(sample_id))
ro.globalenv["adata"] = adata

In [22]:
%%R
sce = devianceFeatureSelection(adata, assay="X")

In [ ]:
binomial_deviance = ro.r("rowData(sce)$binomial_deviance").T
adatas[i], axes2 = hvg(binomial_deviance, axes2, i, adata)

In [ ]:
fig2.tight_layout()
fig2.savefig(os.path.join('ST_normalization_hvg.png'))
fig2.show()

In [25]:
for i in range(len(sample_ids)):
    sample_id = sample_ids[i]
    adata = adatas[i]
    adata.write("adata_{}_scanpy_06052024.h5ad".format(sample_id))